# Mesa Schelling example - Schelling Segregation Model

[[Code explanation]](https://towardsdatascience.com/introduction-to-mesa-agent-based-modeling-in-python-bcb0596e1c9a) **Note that the final interactive visualization part we will cover in the later parts of the course**

## Background

The Schelling (1971) segregation model is a classic of agent-based modeling, demonstrating how agents following simple rules lead to the emergence of qualitatively different macro-level outcomes. Agents are randomly placed on a grid. There are two types of agents, one constituting the majority and the other the minority. All agents want a certain number (generally, 3) of their 8 surrounding neighbors to be of the same type in order for them to be happy. Unhappy agents will move to a random available grid space. While individual agents do not have a preference for a segregated outcome (e.g. they would be happy with 3 similar neighbors and 5 different ones), the aggregate outcome is nevertheless heavily segregated.

# 1. Create the Basic Agent/Model

In [ ]:
from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import SingleGrid

In [ ]:
# Agent

class SchellingAgentBasic(Agent):
    
    def __init__(self, pos, model, agent_type):
        
        super().__init__(pos, model)
        
        # 1 Initialization
        
    def step(self):
        
        # 2 Step agent function
        
        neighbors = self.model.grid.neighbor_iter(self.pos)
        
        for neighbor in neighbors:
            # 3 Calculate the number of similar neighbours
            
        # 4 Move to a random empty location if unhappy
        self.model.grid.move_to_empty(self)

In [ ]:
# Model

class SchellingModelBasic(Model):

    def __init__(self, height, width, density, minority_pc, homophily):
        '''
        Create a new Schelling model.

         Args:
            width: Horizontal axis of the grid which is used together with Height to define the total number of agents in the system.
            height: Vertical axis of the grid which is used together with Width to define the total number of agents in the system.
         '''
        super().__init__()
        
        # 1 Initialization
        
        self.running = True
        
        for cell in self.grid.coord_iter():
            x = cell[1]
            y = cell[2]
            
            # We use a grid iterator that returns the coordinates of a cell as well as its contents. (coord_iter)
            if random.random() < self.density:
                if random.random() < self.minority_pc:
                    agent_type = 1
                else:
                    agent_type = 0
                    
            # 2 Create agents

    def step(self):
        
        self.happy = 0  # 1 Reset counter of happy agents
        
        self.schedule.step()
        
        # 3 Step model function
        
        # 2 Stop the model if all agents are happy
        if self.happy == self.schedule.get_agent_count():
            self.running = False

# 2. Run the Agent/Model Basic

<font color='green'>**HINT:** Now we instantiate a model instance: a 10x10 grid, with an 80% chance of an agent being placed in each cell, approximately 20% of agents set as minorities, and agents wanting at least 3 similar neighbors.</font>

In [ ]:
model = SchellingModelBasic(10, 10, ) # < Add model parameters
for i in range(10):
    model.step()

# 3. Visualize the Agent/Model

In [ ]:
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer


def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 0,
                 "r": 0.5}
    
    if agent.type == 0:
        portrayal["Color"] = "Red"
    else:
        portrayal["Color"] = "Blue"
        
    return portrayal

grid = CanvasGrid(agent_portrayal, 10, 10, 500, 500)
server = ModularServer(SchellingModelBasic,
                       [grid],
                       "Schelling Model",
                       {"width":10, "height":10}) # < Add model parameters

# 4. Run the Agent/Model Visualization

In [ ]:
server.port = 8521 # The default
server.launch()

# 5. Collect data to Analyze the Agent/Model

In [ ]:
# Data collection

def get_model_analysis_data(model):
    
    # 4. Analysis code for collecting interesting data
    
    '''
    Find the % of agents that only have neighbors of their same type.
    '''
    
    for agent in model.schedule.agents:
        for neighbor in model.grid.neighbor_iter(agent.pos):
            if neighbor.type != agent.type:
                # Collect segregated agents number
    
    return interesting_data

In [ ]:
# Agent

class SchellingAgentAnalysis(Agent):
    
    def __init__(self, pos, model):
        
        super().__init__(pos, model)
        
        # 1 Initialization
        
    def step(self):
        
        # 2 Step agent function


In [ ]:
from mesa.datacollection import DataCollector

# Model

class SchellingModelAnalysis(Model):

    def __init__(self, height, width):
        '''
        Create a new Schelling model.

         Args:
            width: Horizontal axis of the grid which is used together with Height to define the total number of agents in the system.
            height: Vertical axis of the grid which is used together with Width to define the total number of agents in the system.
         '''
        super().__init__()
        
        # 1 Initialization

        # 5. Data collector
        self.datacollector = DataCollector(
            {"Model_interesting_data": get_model_analysis_data},  # Model-level collection
            {"x": lambda a: a.pos[0], "y": lambda a: a.pos[1]}) # Agent-level collection
        
        for cell in self.grid.coord_iter():
            x = cell[1]
            y = cell[2]
            
            # We use a grid iterator that returns the coordinates of a cell as well as its contents. (coord_iter)
        
            # 2 Create agents

    def step(self):
        
        self.datacollector.collect(self)
        self.schedule.step()
        
        # 3 Step model function

# 6. Run the Agent/Model Analysis

In [ ]:
import pandas as pd

model_out = model.datacollector.get_model_vars_dataframe()
model_out.head()

In [ ]:
model_out.plot()

In [ ]:
agent_out = model.datacollector.get_agent_vars_dataframe()
agent_out.head()

In [ ]:
agent_out.plot()

# 7. Create iteration Batch of the Agent/Model

In [ ]:
params = {"height": 10, "width": 10, "homophily": range(1,9)} # < Add model parameters

# 8. Run the Agent/Model Batch

In [ ]:
from mesa.batchrunner import batch_run

results = batch_run(
    SchellingModelAnalysis,
    parameters=params,
    iterations=10,
    max_steps=200,
    display_progress=True,
)

# 9. Run the Batch data Analysis

In [ ]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df.head()

Task: Find out how homophily (level of neighbour similaritly) influences the final segragation of agents using the mean or box plot. You should be able to plot the average outcome for each homophily value.

**hint** Your plot should look similar to [this paper](https://www.jasss.org/15/1/6.html). Not neccesarily with Moran's I, but the transition should be visible

In [ ]:
results_df.groupby(by=["RunId"]).median()